# 데이터 불러오기부터

### 가. 라이브러리 Import

In [ ]:
import pandas as pd
import numpy as np

### 나. 데이터 불러오기

### 만약에 Train/Evaluation 데이터를 따로 전처리 한다면?
* 전처리 기준(특히 정규화 작업 시)이 달라져서 모델의 성능에 악영향을 줄 수 있음

In [ ]:
# 데이터 셋 로딩
df = pd.read_csv("onenavi_train.csv", sep = "|")
df_eval = pd.read_csv("onenavi_evaluation.csv", sep = "|")
# 학습/평가 데이터의 전처리 기준을 통일하기위해 데이터 합본
df_total=pd.concat([df,df_eval],ignore_index=True)
# 추가 데이터 셋 로딩
df_pnu = pd.read_csv("onenavi_pnu.csv",sep="|") # 주소(시도/시군구 정보)
df_signal = pd.read_csv("onenavi_signal.csv",sep="|") # 경로의 신호등 갯수
# 데이터 합치기
df_total=pd.merge(df_total,df_pnu , on="RID")
df_total=pd.merge(df_total,df_signal , on="RID")
df_total

---

# 1. 이상치/결측치 처리
## Trash를 판별하는 능력은 좋은 AI모델을 만드는 데 있어 핵심입니다.
+ KeyPoint : 데이터의 형태를 살펴보고 불필요한 데이터를 정제할 수 있다.

[참고] 공식 Document
+ fillna(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)
+ dropna(https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

### 가. 결측치 처리

### 실습1. <u>df_total의 결측치를 확인하고 결측치가 존재한다면 제거해주세요.</u>

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.(Hint : dataframedf_total.isnull().sum() // dropna(), fillna())






### 나. 이상치 처리
#### 이상치(Outlier)는 전적으로 연구자 혹은 개발자가 판단을 해야한다. 자세히 살펴보지않으면 놓치기 쉽상이다.
#### "자세히 보아야 드러난다. 오래 보아야 도움이 된다. Outlier가 그렇다."

### 실습2. <u>시각화 기법 및 데이터 분포를 확인하고 df_total의 이상치를 전처리해주세요.</u>

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.





### 실습3. <u>이상치를 판단한 기준(아이디어)과 처리한 내용을 적어주세요.</u>

In [ ]:
# 아래에 자유롭게 아이디어를 적어주세요. 그리고 실제로 이상치를 처리했다면 처리한 내용을 적어주세요.
# 아이디어 :
# 이상치 처리 :




# 3. 더미변수 생성
## 범주형 데이터도 모델링에 활용할 수 있어야 합니다. 이를 위해 더미변수로 변환해주어야 합니다.
+ KeyPoint : 범주형 변수를 더미변수로 변환할 수 있다.

#### 범주형 변수 중에 어떤 변수를 더미로 변환해볼까요?
#### 요일, 시간, 시도 변수를 더미화 하면 좋을 것 같습니다.

### 실습4. <u>요일, 시간, 날짜 변수를 추가하고 요일, 시간, 날짜, level1_pnu를 더미변수로 만들어봅시다.</u>

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
# Hint1
## for w in df_total['TIME_DEPARTUREDATE']:
##     parse_data_w = parse(w)
##     weekday_list.append(parse_data_w.weekday())
##     hour_list.append(parse_data_w.hour)
##     day_list.append(parse_data_w.day)
# Hint2 : pandas.get_dummies






In [ ]:
# 평가데이터 별도 저장 : 원본 기준(나중에 활용)
new_df_eval=df_total[df_total['DAY']>=27]
new_df_eval.to_csv("onenavi_evaluation_new.csv",sep="|",index=False)

# 4. 데이터 스케일링
##  Feature들의 크기, 범주를 정규화하는 과정을 통해 특정변수의 영향도를 조정해줄 수 있습니다.
+ KeyPoint : 데이터 스케일링을 통해 AI모델 학습에 도움을 줄 수 있다.

### 실습5. <u>df_total을 MinMaxScaler로 데이터 스케일링을 진행하고 feature 변수로 저장해주세요.</u>
* 단, 날짜가 스케일링이 되면 안됩니다. 별도로 저장해주었다가 나중에 합쳐주세요.
* data_day = df_total['DAY']

In [ ]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.(Hint : from sklearn.preprocessing import MinMaxScaler)
# 스케일링이 불가능한 데이터(문자형, 날짜형 등)와 우리의 목적인 Target(ET)은 스케일링을 하시면 안됩니다.






---

## 다음 단계를 위해서 데이터를 저장하겠습니다.

In [ ]:
# 다시 날짜를 합칠께요 : Train/Evaluation 분리를 위해 
feature['DAY']=data_day
# traindata 지정
train_feature=feature[feature['DAY']<=24]
train_feature=train_feature.drop(['DAY'],axis=1)
eval_feature=feature[feature['DAY']>=27]
eval_feature=eval_feature.drop(['DAY'],axis=1)

In [ ]:
len(feature),len(train_feature),len(eval_feature)

In [ ]:
# target 저장
train_target = df_total[df_total['DAY']<=24]['ET']
train_target

In [ ]:
# CSV로 저장
train_feature.to_csv('onenavi_train_feature.csv',index = False,sep='|')
train_target.to_csv('onenavi_train_target.csv',index = False,sep='|')
eval_feature.to_csv('onenavi_eval_feature.csv',index = False,sep='|')